<a href="https://colab.research.google.com/github/Malachyiii/Pawpularity-PytorchLightning-SwinComputerVision/blob/main/PyTorchPawpularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install timm 
!pip install wandb --quiet
!pip install pytorch-lightning

In [ ]:
#Downloading the Data
!wget -O data.zip "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/25383/2684322/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1640884527&Signature=f0IBeyoIypalFjJFpEXnhJ%2BPmRuSKdhfJpkrFb7Paqeesq6ViIeN7SxG%2FfEOq0J%2ByaBA1UoOdsZfFMPHR5GfIXrdEfAZ9JjnFNmnjnG4A7YtxwxzpGibbcMH4etBfmX8dgTTMvPjd0s%2Bcdtdom31gGGYrdzYaV75%2FUkVBD0S7NiL6Dl%2BeWhLzvy7%2FTAPp07WxmWAiGa0pX4ntOTP7iLrczvimj0TQqaeIW1UPIKE6xQM8adbSh53MGHF1MnKdcIrMoP2JWVDT1zUM2uBCcu%2FO5lw%2F2axH7JLtl1UYPSGadSjODnXQZyAVoEa2YlRjj4G7Ze1a1zrXp%2B1qwirfCU7oQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dpetfinder-pawpularity-score.zip"

--2021-12-27 23:30:05--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/25383/2684322/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1640884527&Signature=f0IBeyoIypalFjJFpEXnhJ%2BPmRuSKdhfJpkrFb7Paqeesq6ViIeN7SxG%2FfEOq0J%2ByaBA1UoOdsZfFMPHR5GfIXrdEfAZ9JjnFNmnjnG4A7YtxwxzpGibbcMH4etBfmX8dgTTMvPjd0s%2Bcdtdom31gGGYrdzYaV75%2FUkVBD0S7NiL6Dl%2BeWhLzvy7%2FTAPp07WxmWAiGa0pX4ntOTP7iLrczvimj0TQqaeIW1UPIKE6xQM8adbSh53MGHF1MnKdcIrMoP2JWVDT1zUM2uBCcu%2FO5lw%2F2axH7JLtl1UYPSGadSjODnXQZyAVoEa2YlRjj4G7Ze1a1zrXp%2B1qwirfCU7oQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dpetfinder-pawpularity-score.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 173.194.211.128, 173.194.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1030728211 (983M) [application/zip]
Saving to: ‘data.zip’

dat

In [ ]:
! unzip -qq "data.zip" -d data

replace data/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR

import torchvision
from torchvision import transforms as T
from torchvision.io import read_image

import timm

from tqdm import tqdm_notebook as tqdm

import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import progress
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor

import sklearn
from sklearn.model_selection import StratifiedKFold

import wandb

In [ ]:
#timm.list_models(pretrained=True)

In [ ]:
class args:
  #Overall Args
  folder_name = "data"
  pretrained = True
  imagesize = 224 
  model_name = 'swin_small_patch4_window7_224' #'cait_m36_384'
  num_classes = 1
  num_workers = 4
  model_shape = 768 #1536 for swin large 1792 for efficientnet b4 768 for cait-m-36
  n_splits = 10

  #Training Args
  train_batch_size = 64
  val_batch_size = 64
  test_batch_size = 64
  weight_decay = 1e-6
  max_epochs = 15
  
  #Optimizer and Scheduler args
  loss = 'nn.BCEWithLogitsLoss'
  lr = 1e-4
  warmup_epochs = 0
  weight_decay = 1e-6
  eta_min = 0.0000001
  n_accumulate = 1
  T_0 = 25
  T_max = 100

  #Callback args
  min_delta = 0.0
  patience = 10

#Dataloader Args
loaderargs = {'num_workers' : args.num_workers, 'pin_memory': False, 'drop_last': False}
device = torch.device("cuda:0")


In [ ]:
df = pd.read_csv(f'/content/{args.folder_name}/train.csv')
df['filename'] = f'/content/{args.folder_name}/train/' + df['Id'] + '.jpg'

feature_cols = [col for col in df.columns if col not in ['Id', 'Pawpularity', 'filename']]

In [ ]:
class PawpularityDataset(torch.utils.data.Dataset):
    def __init__(self, df, test = False):
        self.df = df
        self.file_names = df['filename'].values
        self.targets = df['Pawpularity'].values
        self._transform = T.Resize(size= (args.imagesize, args.imagesize))
        self.test = test
        self.meta = df[feature_cols].values

        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = read_image(img_path)
        img = self._transform(img)

        meta = self.meta[index, :]
        
        if self.test:
          target = 0
        else:
          target = self.targets[index]
            
        return img, target, meta

In [ ]:
class PawpularityDataModule(pl.LightningDataModule):
    def __init__(self, traindf, valdf,args, loaderargs):
        super().__init__()
        self._train_df = traindf
        self._val_df = valdf
        self.args = args
        self.loaderargs = loaderargs


    def __create_dataset(self, train=True):
        if train == 'train':
          return PawpularityDataset(self._train_df)
        else:
          return PawpularityDataset(self._val_df)


    def train_dataloader(self):
        pawpularity_train = self.__create_dataset("train")
        return DataLoader(pawpularity_train, **self.loaderargs, batch_size=self.args.train_batch_size)

    def val_dataloader(self):
        pawpularity_val = self.__create_dataset("val")
        return DataLoader(pawpularity_val, **self.loaderargs, batch_size=self.args.val_batch_size)
    

In [ ]:
def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean = (0.485, 0.456, 0.406), 
                            std = (0.229, 0.224, 0.225))
                
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean = (0.485, 0.456, 0.406), 
                            std = (0.229, 0.224, 0.225))
            ]
        ),
    }
    return transform
  

def mixup(x: torch.Tensor, y: torch.Tensor, z = torch.Tensor, alpha: float = 1.0):
    assert alpha > 0, "alpha should be larger than 0"
    assert x.size(0) > 1, "Mixup cannot be applied to a single instance."

    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x.size()[0])
    mixed_x = lam * x + (1 - lam) * x[rand_index, :]
    mixed_meta = lam * z + (1 - lam) * z[rand_index, :]
    target_a, target_b = y, y[rand_index]
    return mixed_x, mixed_meta, target_a, target_b,  lam

In [ ]:
#Use this to find the model_shape attribute when changing models

#x = torch.randn(1,3,args.imagesize,args.imagesize)
#model = timm.create_model(args.model_name, #                                       pretrained=args.pretrained, 
#                                      num_classes=0,
#                                       in_chans = 3)
#model(x).shape

In [ ]:
class DenseModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.args = args
        self.model = timm.create_model(args.model_name, 
                                       pretrained=args.pretrained, 
                                       num_classes=0,
                                       in_chans = 3)
        self.fc1 = nn.Linear(args.model_shape, 128)
        self.fc2 = nn.Linear(140, 64)
        self.fc3 = nn.Linear(64, args.num_classes)
        self.dropout = nn.Dropout(p=0.1)
        self._criterion = eval(self.args.loss)()
        self.transform = get_default_transforms()

    def forward(self, features, meta):
        features = self.model(features)                 
        features = self.dropout(features)
        features = self.fc1(features)
        features = torch.cat([features, meta], dim=1)
        features = self.fc2(features)
        output = self.fc3(features)           
        return output

    def __share_step(self, batch, mode):
        images, labels, meta = batch
        labels = labels.float() / 100.0
        meta = meta.float()
        images = self.transform[mode](images)

        if torch.rand(1)[0] < 0.5 and mode == 'train':
            mix_images, mixed_meta, target_a, target_b, lam = mixup(images, labels, meta, alpha=0.5)
            logits = self.forward(mix_images, mixed_meta).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + \
                (1 - lam) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images, meta).squeeze(1)
            loss = self._criterion(logits, labels)

        pred = logits.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        
        return loss, pred, labels

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        return {'loss': loss, 'pred': pred, 'labels': labels}



    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return {'pred': pred, 'labels': labels}


    def training_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'train')

    def validation_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'val')

        
    def __share_epoch_end(self, outputs, mode):
        preds = []
        labels = []
        for out in outputs:
            pred, label = out['pred'], out['labels']
            preds.append(pred)
            labels.append(label)
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        metrics = torch.sqrt(((labels - preds) ** 2).mean())
        self.log(f'{mode}_RMSE', metrics)    


    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=args.lr, weight_decay = args.weight_decay)
        return {
        "optimizer": optimizer,
        "lr_scheduler": {
            "scheduler": CosineAnnealingLR(optimizer, T_max = args.T_max, eta_min= args.eta_min),
            "interval": "step",
            "monitor": "train_loss",
            "frequency": 1}
            }

In [ ]:
Kfolds = StratifiedKFold(n_splits=args.n_splits, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(Kfolds.split(df["Id"], df["Pawpularity"])):
    traindf = df.loc[train_idx].reset_index(drop=True)
    valdf = df.loc[val_idx].reset_index(drop=True)

    
    model = DenseModel()

    #Callbacks
    early_stop_callback = EarlyStopping(monitor="val_RMSE", min_delta=args.min_delta, patience=args.patience, 
                                        verbose=False, mode="min")
    progressbar = TQDMProgressBar(refresh_rate = 10)
    checkpoint_callback = ModelCheckpoint(dirpath="/content/drive/MyDrive/PawpularityWeights/", 
                                          filename= f"64_{fold}best_weights", save_top_k=1, monitor="val_RMSE")
    lr_monitor = LearningRateMonitor(logging_interval='step')

    wandb_logger = WandbLogger(project='lightning_test', job_type='train', log_model = 'all')

    wandb_logger.watch(model)

    trainer = pl.Trainer(max_epochs=args.max_epochs, 
                        gpus=1, 
                        logger=wandb_logger,
                        callbacks=[early_stop_callback, 
                                    progressbar, 
                                    checkpoint_callback,
                                    lr_monitor])

    Pawpularity_Datamodule = PawpularityDataModule(traindf, valdf, args = args, loaderargs = loaderargs)

    trainer.fit(model, Pawpularity_Datamodule)

    run_id = trainer.logger.experiment.id
    project = trainer.logger.experiment.project
    entity = trainer.logger.experiment.entity

    del model
    torch.cuda.empty_cache()

    wandb.finish()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
wandb: Currently logged in as: malachyiii (use `wandb login --relogin` to force relogin)


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | model      | SwinTransformer   | 48.8 M
1 | fc1        | Linear            | 98.4 K
2 | fc2        | Linear            | 9.0 K 
3 | fc3        | Linear            | 65    
4 | dropout    | Dropout           | 0     
5 | _criterion | BCEWithLogitsLoss | 0     
-------------------------------------------------
48.9 M    Trainable params
0         Non-trainable params
48.9 M    Total params
195.779   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:686: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


wandb: ERROR Control-C detected -- Run data was not synced


In [ ]:
#script = testmodel.to_torchscript()
#torch.jit.save(script, "model.ts")



1.   clean-galaxy-13 test RMSE 18.8
    -  Augmentations
            
            T.Normalize(mean = (0.485, 0.456, 0.406), 
                        std = (0.229, 0.224, 0.225)),
            T.RandomHorizontalFlip(),
            T.Resize(size= (args.imagesize, args.imagesize))]
    - class args:
 
 
      folder_name = 'data'
      imagesize = 512
      model_name = 'tf_efficientnet_b4_ns'
      pretrained = True
      num_classes = 1
      num_workers = 4
      model_shape = 1792

      train_batch_size = 4
      val_batch_size = 4
      test_batch_size = 4
      weight_decay = 1e-6
  
      lr = 1e-4
      warmup_epochs = 0
      weight_decay = 1e-6
      eta_min = 0.0000001
      n_accumulate = 1
      T_0 = 25
      T_max = 100

2.     distinctive-fog-17 test RMSE: diverged
    -  Added Augmentations
           
           
            T.RandomVerticalFlip(),
            T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
            T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1)


3.      distinctive-fog-17 test RMSE: about the same as clean-galaxy-13 (no improvement
    -  Removed Augmentations affine and jitter leaving only
           
           
            T.Normalize(mean = (0.485, 0.456, 0.406), 
                        std = (0.229, 0.224, 0.225)),
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.Resize(size= (args.imagesize, args.imagesize))]
            
4. Widened Lr: Worse